## Check Equilibrium
This notebook reads files describing a structure, and the files output by Frame2D after an
analysis, and checks that the forces and moments on every node are in equilibrium.

It does this in the simplest way possible, using quite different logic than Frame2D, resulting
in a higher degree of confidence in the results.  It would have been better had someone else
programmed it, but oh well ...

In [1]:
dir = 'KG82sd.d'
dir = 'l22x6.d'
dir = 'l22x6pd.d'

def filename(basename):
    return dir + '/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [2]:
import pandas as pd
import math

In [3]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.sumFX = 0.
        self.sumFY = 0.
        self.sumMZ = 0.

In [4]:
table = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in table.iterrows():
    if n.ID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.ID))
    NODES[n.ID] = Node(n.ID,float(n.X),float(n.Y))

In [5]:
class Member(object):
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L

In [6]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.ID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.ID))
    MEMBERS[m.ID] = Member(m.ID,NODES[m.NODEJ],NODES[m.NODEK])

In [7]:
table = pd.read_csv(filename('node_loads'))
for i,p in table.iterrows():
    dirn = p.DIRN.upper()
    if dirn in ['FX','FY','MZ']:
        n = NODES[p.ID]
        a = 'sum'+dirn
        setattr(n,a,getattr(n,a,0.)+float(p.F))
    else:
        Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))

In [8]:
try:
    table = pd.read_csv(filename('pdelta_forces'))
    for i,p in table.iterrows():
        dirn = p.DIRN.upper()
        if dirn in ['FX','FY','MZ']:
            n = NODES[p.ID]
            a = 'sum'+dirn
            setattr(n,a,getattr(n,a,0.)+float(p.F))
        else:
            Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))
except OSError:
    pass

In [9]:
table = pd.read_csv(filename('reactions'))
for i,r in table.iterrows():
    n = NODES[r.ID]
    n.sumFX += 0. if pd.isnull(r.FX) else float(r.FX)
    n.sumFY += 0. if pd.isnull(r.FY) else float(r.FY)
    n.sumMZ += 0. if pd.isnull(r.MZ) else float(r.MZ)

In [10]:
mtable = pd.read_csv(filename('mefs'))
for i,row in mtable.iterrows():
    m = MEMBERS[row.ID]
    n = m.nodej
    n.sumFX -= row.FXJ*m.cosx - row.FYJ*m.cosy
    n.sumFY -= row.FXJ*m.cosy + row.FYJ*m.cosx
    n.sumMZ -= row.MZJ
    n = m.nodek
    n.sumFX -= row.FXK*m.cosx - row.FYK*m.cosy
    n.sumFY -= row.FXK*m.cosy + row.FYK*m.cosx
    n.sumMZ -= row.MZK
mtable

,ID,FXJ,FYJ,MZJ,FXK,FYK,MZK
0,CA0A1,542068.693854,944801.389259,3.345741e+09,-542068.693854,-944801.389259,2.795468e+09
1,CB0B1,16222731.306100,995542.969518,3.458844e+09,-16222731.306100,-995542.969518,3.012185e+09
2,CC0C1,3631365.956540,930121.493961,3.322237e+09,-3631365.956540,-930121.493961,2.723553e+09
3,CD0D1,11395838.401000,278203.458179,-2.980232e-08,-11395838.401000,-278203.458179,1.808322e+09
4,CE0E1,17420795.642500,218004.089743,-5.960464e-08,-17420795.642500,-218004.089743,1.417027e+09
5,CF0F1,3284831.881230,964089.569783,3.399037e+09,-3284831.881230,-964089.569783,2.867545e+09
6,CG0G1,13479968.118800,1009122.441770,3.494127e+09,-13479968.118800,-1009122.441770,3.065168e+09
7,CA1A2,1135902.326860,531779.543827,1.417337e+09,-1135902.326860,-531779.543827,1.507450e+09
8,CB1B2,14860097.673100,646815.002044,1.744463e+09,-14860097.673100,-646815.002044,1.813020e+09
9,CC1C2,3839216.130240,501695.281295,1.289995e+09,-3839216.130240,-501695.281295,1.469329e+09


In [11]:
sums = pd.DataFrame([(n.id,n.sumFX,n.sumFY,n.sumMZ) for n in NODES.values()],
                    columns=['ID','sumFX','sumFY','sumMZ']).set_index(['ID'])
sums

,sumFX,sumFY,sumMZ
ID,,,
C6,7.199706e-07,-4.000642e-06,0.000000e+00
E5,-1.000881e-07,4.000030e-05,0.000000e+00
E8,-5.000111e-07,-3.800134e-05,0.000000e+00
B12,-9.000360e-07,8.000410e-06,2.384186e-07
C3,-4.131143e-07,-7.000053e-06,0.000000e+00
A0,0.000000e+00,0.000000e+00,0.000000e+00
A8,2.800225e-07,8.400457e-06,4.999876e-03
A4,-3.800087e-07,2.328306e-10,-1.000023e-02
B1,1.000008e-06,-1.000054e-05,-9.999275e-03


In [12]:
sums.abs().max()

sumFX    0.000004
sumFY    0.000087
sumMZ    0.010000
dtype: float64

In [13]:
from numpy.linalg import norm

In [14]:
norm(sums.sumFX,2)/min(norm(mtable.FXJ,2),norm(mtable.FXK,2))

1.2277130763792073e-13

In [15]:
norm(sums.sumFY,2)/min(norm(mtable.FYJ,2),norm(mtable.FYK,2))

3.9395345854394779e-11

In [16]:
norm(sums.sumMZ,2)/min(norm(mtable.MZJ,2),norm(mtable.MZK,2))

2.3490695551644504e-12

In [17]:
norm(mtable.FXJ,2),norm(mtable.FXK,2),norm(mtable.FXJ,2)/norm(mtable.FXK,2)

(78319597.650738388, 78319597.650738388, 1.0)

In [18]:
norm(mtable.FYJ,2),norm(mtable.FYK,2),norm(mtable.FYJ,2)/norm(mtable.FYK,2)

(5743697.1669267807, 7754792.3419856597, 0.74066421299633067)

In [19]:
norm(mtable.MZJ,2),norm(mtable.MZK,2), norm(mtable.MZJ,2)/norm(mtable.MZK,2)

(21496408161.744377, 24868315486.718582, 0.86440950024238516)